Repita o exercício 2 mas em vez de parar a simulação depois de gerar N clientes, a simulação vai parar aplicando a regra de parada de Chow e Robbins.

Para isso, N deve crescer indefinidamente e a simulação deve parar quando o intervalo de confiança alcançar um determinado tamanho 2d. Registre o valor final de N. Faça esse experimento para 3 valores de d:

In [17]:
import random                      
import simpy # biblioteca de simulação
import math as mt
from tqdm import tqdm
import statistics as ss
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np


In [18]:
taxaEntrada = 9           # taxa de entrada: λ=9 clientes por segundo. 
taxaServico = 10          # taxa de serviço: μ=10 clientes por segundo.

In [19]:
def media(lista):
    soma = 0
    for i in range(len(lista)):
        soma += lista[i]
    return soma/len(lista)

def varAmostral(lista):
    m = media(lista)
    soma = 0
    for i in range(len(lista)):
        soma += (lista[i]-m)**2
    return soma/(len(lista)-1)
      
def desPadrao(lista):
    dp = mt.sqrt(varAmostral(lista))
    return dp

def exponencial(b):
    u = random.random()
    x = -b * mt.log(1-u)

    return(x)

In [21]:
def tempoQuantiMedia(quanti, media):
    return [-(1/media) *  np.log(1 - np.random.uniform(0,1)) for _ in range(quanti-1)]

In [22]:
def mm1(n, lamb, mi):
    medioEsperaFila = tempoQuantiMedia(n, lamb)
    servicos = tempoQuantiMedia(n, mi)
    filaEsperas = [0]
    espera = 0
    i = 0
    while i < len(medioEsperaFila):
        servicoCompleto = servicos[i] + filaEsperas[-1]
        if (medioEsperaFila[i] <= servicoCompleto):
             espera = (servicos[i] - medioEsperaFila[i]) + filaEsperas[-1]
        else:
             espera = 0
        filaEsperas.append(espera)
        i+=1
    return (filaEsperas)

In [23]:
def mser5y(nClientes, x):
    auxList = []
    aux = 0
    z = []
    i = j =0
    while i < nClientes:
        while j < 5:
            auxList.append(x[i])
            i +=1
            j +=1
        z.append(media(auxList))
        auxList.clear()
        j=0
    mser5yList = []
    k = nClientes/5
    meio = k/2
    d = 0
    while d < meio:
        desvpZ = ss.fmean(z)
        aux = desvpZ / mt.sqrt(k-d)
        mser5yList.append(aux)
        d += 1
        del z[0]
    menor = min(mser5yList)              
    posicao = mser5yList.index(menor)
    return posicao

In [28]:
def truncagem(nClientes):
    i =0
    n = nClientes
    x = mm1(n, 9, 10) # 10^5
    mser5yList =  mser5y(n, x)

    while mser5yList == 0:
        print('Ainda não se encontrou o ponto de truncagem.\n')
        print('A coletar mais informações até que se encontre...')
        n += 2000  
        x = mm1(n, 9, 10)
        mser5yList = mser5y(n, x)

    print('Encontrou o fim do transiente...')
    print('Eliminando as observações do estado transiente...')
    print('Tamanho de M/M/1 final:', len(x))
    pontoTruc = mser5yList * 5
    print(f'Encontrou-se o ponto de truncagem. Na posição {pontoTruc} da fila')
    #eliminando transiente
    while i < pontoTruc:
        del x[0]
        i +=1

    print('Tamanho de M/M/1 após o corte: ', len(x), '\nAgora está no estado estacionario.\n')
    
    wbarra = sum(x)/len(x)
    print('Wbarra: ', wbarra)
    
    z = 1.96 * desPadrao(x)/mt.sqrt(n)
    
    interaloPositivo = wbarra + z
    interaloNegativo = wbarra - z
    
    print(f'Intervalo de confiança: ({interaloPositivo}, {interaloNegativo})', )
    
    return wbarra

In [53]:
def regraParada(n, d):
    print(f"Simulação com d = {d}...")
    nFinal = truncagem(n)
    while nFinal < 2*d:
        print('Aumento do N...')
        nFinal += truncagem(n)
        print(nFinal)
    print(f'Parada da simulação com d = {d}...')
    return nFinal
    
    



In [54]:
valorN1 = regraParada(1000, 1) # d=1

Simulação com d = 1...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 3000
Encontrou-se o ponto de truncagem. Na posição 1455 da fila
Tamanho de M/M/1 após o corte:  1545 
Agora está no estado estacionario.

Wbarra:  0.5698004995141887
Intervalo de confiança: (0.5909834198454719, 0.5486175791829054)
Aumento do N...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 7000
Encontrou-se o ponto de truncagem. Na posição 1060 da fila
Tamanho de M/M/1 após o corte:  5940 
Agora está no estado estaciona

In [55]:
valorN2 = regraParada(1000, 2) # d=2

Simulação com d = 2...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 5000
Encontrou-se o ponto de truncagem. Na posição 400 da fila
Tamanho de M/M/1 após o corte:  4600 
Agora está no estado estacionario.

Wbarra:  0.745981663702718
Intervalo de confiança: (0.7654610564881543, 0.7265022709172817)
Aumento do N...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 5000
Encontrou-se o ponto de truncagem. Na posição 1020 da fila
Tamanho de M/M/1 após o corte:  3980 
Agora está no estado estacionari

In [56]:
valorN4 = regraParada(1000, 4) # d=2

Simulação com d = 4...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 7000
Encontrou-se o ponto de truncagem. Na posição 1895 da fila
Tamanho de M/M/1 após o corte:  5105 
Agora está no estado estacionario.

Wbarra:  0.5920472548567259
Intervalo de confiança: (0.6055856827868936, 0.5785088269265583)
Aumento do N...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 1000
Encontrou-se o ponto de truncagem. Na posição 495 da fila
Tamanho de M/M/1 após o corte:  505 
Agora está no estado estacionario.

Wbarra:  0.4750099540265264
Intervalo de confiança: (0.5009180272539405, 0.4491018807991123)

In [57]:
print(f'Valor final n com d1: {valorN1}')
print(f'Valor final n com d2: {valorN2}')
print(f'Valor final n com d4: {valorN4}')

Valor final n com d1: 2.404871215999662
Valor final n com d2: 4.209712285229738
Valor final n com d4: 8.342717473191888
